In [6]:
#Import Dependencies
import pandas as pd
import numpy as np

#Import matplotlib dependencies
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress

import json 

import geopandas as gpd

#File to load
file_path = "./Data/minwage.csv"

ModuleNotFoundError: No module named 'geopandas'

In [4]:
#Create dataframe
min_wage_df = pd.read_csv(file_path,encoding='cp1252') 
min_wage_df.head()


NameError: name 'file_path' is not defined

In [5]:
min_wage_df['State'].unique()

NameError: name 'min_wage_df' is not defined

In [ ]:
# Show column names 
for col in min_wage_df.columns: 
    print(col) 

In [ ]:
clean_min_wage_df = min_wage_df[['Year', 'State', 'State.Minimum.Wage', 'Federal.Minimum.Wage']]
clean_min_wage_df.head(10)

In [ ]:
# Drop earlier years

drop_early_df = clean_min_wage_df[clean_min_wage_df['Year'] >= 1970]
drop_early_df.head(10)


In [ ]:
#Load in Latlngs
latlong_file_path = "./Data/statelatlong.csv"
latlong_df = pd.read_csv(latlong_file_path,encoding='cp1252') 
latlong_df = latlong_df.drop('State', 1)
latlong_df = latlong_df.rename(columns={'City': 'State'})
latlong_df.head()

In [ ]:
latlong_df['State'].unique()

In [ ]:
minwage_with_latlng = pd.merge(drop_early_df,  
                     latlong_df,  
                     on ='State')

minwage_with_latlng

In [ ]:
len(set(minwage_with_latlng['State'].values))

In [ ]:
print(minwage_with_latlng["Year"].max())
print(minwage_with_latlng['Year'].min())

In [ ]:
# Create bins in which to place values based upon TED Talk views
bins = [1969, 1979, 1989, 1999, 2009, 2019,2020 ]

# Create labels for these bins
group_labels = ['1970',"1980", "1990", "2000", "2010",'2020']

In [ ]:
# Place the data series into a new column inside of the DataFrame
minwage_with_latlng["Decade"] = pd.cut(minwage_with_latlng["Year"], bins, labels=group_labels)
minwage_with_latlng.sample(20)

In [ ]:
# Create a GroupBy object based upon "View Group"


statistics_df=round(minwage_with_latlng.groupby(['Decade', 'State']).agg(['mean', 'median'])['State.Minimum.Wage'],2)
statistics_df=statistics_df.reset_index()
statistics_df

In [ ]:
statistics_df_federal=round(minwage_with_latlng.groupby(['Decade', 'State']).agg(['mean', 'median'])['Federal.Minimum.Wage'],2)
statistics_df_federal=statistics_df_federal.reset_index()
statistics_df_federal

In [ ]:
statistics_df

In [ ]:
minwage_with_latlng.info()

In [ ]:
minwage_with_latlng['Decade']= minwage_with_latlng['Decade'].astype(str)
minwage_with_latlng.info()

In [ ]:
#del minwage_with_latlng["Year"]

In [ ]:
# print('We have {} geotagged rows'.format(len(minwage_with_latlng)))

In [ ]:
# def df_to_geojson(minwage_with_latlng, properties, lat='Latitude', lon='Longitude'):
#     """
#     Turn a dataframe containing point data into a geojson formatted python dictionary
    
#     df : the dataframe to convert to geojson
#     properties : a list of columns in the dataframe to turn into geojson feature properties
#     lat : the name of the column in the dataframe that contains latitude data
#     lon : the name of the column in the dataframe that contains longitude data
#     """
    
#     # create a new python dict to contain our geojson data, using geojson format
#     geojson = {'type':'FeatureCollection', 'features':[]}

#     # loop through each row in the dataframe and convert each row to geojson format
#     for _, row in minwage_with_latlng.iterrows():
#         # create a feature template to fill in
#         feature = {'type':'Feature',
#                    'properties':{},
#                    'geometry':{'type':'Point',
#                                'coordinates':[]}}

#         # fill in the coordinates
#         feature['geometry']['coordinates'] = [row[lon],row[lat]]

#         # for each column, get the value and add it as a new feature property
#         for prop in properties:
#             feature['properties'][prop] = row[prop]
        
#         # add this feature (aka, converted dataframe row) to the list of features inside our dict
#         geojson['features'].append(feature)
    
#     return geojson

In [ ]:
# useful_columns = ['Year','State','State.Minimum.Wage','State.Minimum.Wage.2020.Dollars','Federal.Minimum.Wage', 'Federal.Minimum.Wage.2020.Dollars','Latitude','Longitude','Decade']
# geojson_dict = df_to_geojson(minwage_with_latlng, properties=useful_columns)
# geojson_str = json.dumps(geojson_dict, indent=2)
# print(geojson_str)

In [ ]:
# Can't use this, <class 'str'> type

In [ ]:
geojson_internet=gpd.read_file('gz_2010_us_040_00_500k.json')
geojson_internet

In [ ]:
renamed_df=geojson_internet.rename(columns = {'STATE':'Name', 'NAME':'State'})
renamed_df

In [ ]:
geojson_data=renamed_df.merge(minwage_with_latlng, on='State')
geojson_data

In [ ]:
geojson_data_final=geojson_data.merge(statistics_df, on=['Decade','State'])
geojson_data_final

In [ ]:
geojson_data_final=geojson_data_final.merge(statistics_df_federal, on=['Decade','State'])
geojson_data_final

In [ ]:
geojson_data_to_upload = geojson_data_final.rename(columns = {'mean_x':'State_Average_by_Decade', 'median_x':'State_Median_by_Decade','mean_y':'Federal_Average_by_Decade', 'median_y':'Federal_Median_by_Decade'})
geojson_data_to_upload

In [ ]:
# # rows = []

# for ii in range(1970, 2021):
#     test1 = geojson_data_to_upload[geojson_data_to_upload.Year == ii]

# # print(test1)
#     test1.to_file("./map/static/data/"+str(ii)+".geojson", driver='GeoJSON')
# # for ii , row in geojson_data_to_upload.iterrows():
# #     if(row.Year == 1970):
# #         rows.append(row)

In [ ]:
print(type(geojson_data_to_upload))

In [ ]:
geojson_data_to_upload.to_file("minwage.geojson", driver='GeoJSON')

In [ ]:
geojson_internet2=gpd.read_file('us_states.geojson')
geojson_internet2

In [ ]:
renamed_df=geojson_internet2.rename(columns = {'name':'State'})
renamed_df

In [ ]:
geojson_data=renamed_df.merge(minwage_with_latlng, on='State')
geojson_data

In [ ]:
geojson_data_final=geojson_data.merge(statistics_df, on=['Decade','State'])
geojson_data_final

In [ ]:
geojson_data_final = geojson_data_final.merge(statistics_df_federal, on=['Decade','State'])
geojson_data_final

In [ ]:
geojson_data_to_upload=geojson_data_final.rename(columns = {'mean_x':'State_Average_by_Decade', 'median_x':'State_Median_by_Decade','mean_y':'Federal_Average_by_Decade', 'median_y':'Federal_Median_by_Decade'})
geojson_data_to_upload

In [ ]:
geojson_data_to_upload.to_file("minwage2try.geojson", driver='GeoJSON')